In [1]:
import re
import time

import numpy as np
import pandas as pd
import requests

In [2]:
# !pip3 install progressbar2
from progressbar import progressbar

## Scraping

### Scraping Functions

In [3]:
# function to scrape reddit page (takes a reddit .json url)
# returns posts 

def scraper_bike(url):
    headers = {'User-Agent' : 'override this bad boy!'}
    posts = []
    after = {}

    for page in progressbar(range(40)):
        params = {'after': after}
        pagepull = requests.get(url=url, params=params, headers=headers)
        page_dict = pagepull.json()
        posts.extend(page_dict['data']['children'])
        after = page_dict['data']['after']
        time.sleep(.2)
        
    return posts

In [4]:
# function to convert posts to DataFrame - won't allow duplicate posts since unique id 'name' is set as index
# Extract: name (as index) and subreddit, selftext, title (as columns)

def posts_to_df(post_list):
    i = 0
    post_dict = {}
    
    for post in post_list:
        ind = post_list[i]['data']
        post_dict[ind['name']] = [ind['subreddit'], ind['title'], ind['selftext']]
        i += 1

    df_name = pd.DataFrame(post_dict)
    df_name = df_name.T
    df_name.columns = ['subreddit', 'title', 'selftext']
    
    return df_name

In [5]:
# takes scraper function and url - outputs dataframe

def scrape_to_df(scrape_func, url):
    
    return posts_to_df(scrape_func(url))

In [6]:
#### If you want to scrape repeatedly over time and add to a csv
# scrape, import csv, concat, drop duplicate, and output to csv
# takes in scraper function, url, csv filename to import, csv filename to output
# Outputs - Concatenated DataFrame as csv

def scrape_add(scrape_func, url, import_file, export_file):
    scrape_df = posts_to_df(scrape_func(url))
    imported_df = pd.read_csv(import_file, index_col = 'Unnamed: 0')
    concat_df = pd.concat([imported_df, scrape_df])
    concat_df = concat_df[~concat_df.index.duplicated(keep='first')]
    concat_df.to_csv(export_file)

### Run Scrape

In [7]:
# You can also put in any 2 subreddits in as the URL and get results for those

nfltest = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/nfl.json')
nbatest = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/nba.json')

100% (40 of 40) |########################| Elapsed Time: 0:00:29 Time:  0:00:29
100% (40 of 40) |########################| Elapsed Time: 0:00:28 Time:  0:00:28


In [8]:
politics_test = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/politics.json')
conservative_test = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/conservative.json')

100% (40 of 40) |########################| Elapsed Time: 0:00:31 Time:  0:00:31
100% (40 of 40) |########################| Elapsed Time: 0:00:28 Time:  0:00:28


In [9]:
nbatest.shape

(705, 3)

In [10]:
nfltest.shape

(872, 3)

##### These scrape_add functions add to already built csvs

In [11]:
# scrape_add(scraper_bike, 'https://www.reddit.com/r/CollegeBasketball/new.json', 'NCAA_Posts_Update2.csv', 'NCAA_Posts_Update3.csv')
# scrape_add(scraper_bike, 'https://www.reddit.com/r/AskScience/new.json', 'AskSci_Posts_Update2.csv', 'AskSci_Posts_Update3.csv')
# scrape_add(scraper_bike, 'https://www.reddit.com/r/nba/new.json', 'NBA_Posts_Update2.csv', 'NBA_Posts_Update3.csv')
# scrape_add(scraper_bike, 'https://www.reddit.com/r/nfl/new.json', 'NFL_Posts_Update2.csv', 'NFL_Posts_Update3.csv')

### Data Cleaning / Preprocessing

In [12]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

pd.set_option('max_colwidth', 300)

In [13]:
# drop column

nfltest = nfltest.drop(columns = 'selftext')
nbatest = nbatest.drop(columns = 'selftext')

In [14]:
# merge subreddit data

train = pd.concat([nfltest, nbatest])

In [15]:
train

,subreddit,title
t3_hrq7wi,nfl,Water Cooler Wednesday
t3_hrryev,nfl,Offseason Review Series: Day 1 - The Cincinnati Bengals
t3_hro3br,nfl,[Garafola] WR Kenny Stills arrested with felony charge after peaceful demonstration
t3_hrqyzt,nfl,[Ryan Williams] Kenny Stills: - stood on a lawn - asked for justice - got arrested Cops that killed Breonna Taylor: - kicked in her door - killed her - didn't get arrested @KSTiLLS has officially spent more time in jail fighting for justice than the known killers who killed Breonna Taylor.
t3_hrmkgq,nfl,"[Heitner] Jameis Winston has moved on from NFL agent Joel Segal. @Jaboowins' next agent, if he hires one, will be his 3rd since entering the league as the No. 1 overall pick in 2015."
...,...,...
t3_hous5j,nba,Blake Griffin puts Kendrick Perkins in a body bag!
t3_hp4al2,nba,Who's got YOUR favorite jump shot in the NBA? Smoothest J in the league?
t3_hop1p4,nba,"[Windhorst] As the Disney bubble gets set up and tries to hold together, NBA teams are quietly working on another complex problem. How to construct next season with huge virus &amp; money challenges (w/@TimBontemps)"
t3_hpfo2h,nba,Middleton or DeRozan? Who would be the better #1 option?


##### Tokenize (grab only word characters)

In [16]:
word_tokenizer = RegexpTokenizer(r'\w+')

In [17]:
train['title'] = train['title'].map(lambda x: word_tokenizer.tokenize(x.lower()))

In [18]:
# rejoin list of tokenized words into single string for each row

train['title'] = train['title'].map(lambda x: ' '.join(x))

In [19]:
train['title'][0:5]

t3_hrq7wi                                                                                                                                                                                                                                                              water cooler wednesday
t3_hrryev                                                                                                                                                                                                                                offseason review series day 1 the cincinnati bengals
t3_hro3br                                                                                                                                                                                                   garafola wr kenny stills arrested with felony charge after peaceful demonstration
t3_hrqyzt    ryan williams kenny stills stood on a lawn asked for justice got arrested cops that killed breonna taylor kicked in her door kill

In [20]:
train['title'][0:5]

t3_hrq7wi                                                                                                                                                                                                                                                              water cooler wednesday
t3_hrryev                                                                                                                                                                                                                                offseason review series day 1 the cincinnati bengals
t3_hro3br                                                                                                                                                                                                   garafola wr kenny stills arrested with felony charge after peaceful demonstration
t3_hrqyzt    ryan williams kenny stills stood on a lawn asked for justice got arrested cops that killed breonna taylor kicked in her door kill

### Train test split and converting series to list of strings then to array

In [21]:
X = train[['title']]
y = train['subreddit']

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=.25,
                                                    random_state=42,
                                                    stratify=y)

In [23]:
# baseline is

y.value_counts(normalize=True)

nfl    0.552949
nba    0.447051
Name: subreddit, dtype: float64

In [24]:
# create our training data list - this is a list of strings, with each string being a post title

clean_train_data = []

for traindata in X_train['title']:
    clean_train_data.append(traindata)

In [25]:
len(clean_train_data)

1182

In [26]:
# create test data list

clean_test_data = []

for testdata in X_test['title']:
    clean_test_data.append(testdata)

In [27]:
len(clean_test_data)

395

### Count Vectorizer

In [28]:
# instantiate our CountVectorizer. This counts the number of appearances of all the words in our training data and
# eliminates common english stop words. 5000 max features works well for our purposes (tested various numbers). Our
# data is already preprocessed and tokenized manually earlier. ngram_range is 1,3, although all or nearly all our
# features are single words

vectorizer = CountVectorizer(analyzer="word",
                             tokenizer=None,
                             preprocessor=None,
                             stop_words='english',
                             max_features=5000,
                             ngram_range=(1, 3))

In [29]:
# fit our training data and test data lists to our count_vectorizer

train_data_features = vectorizer.fit_transform(clean_train_data)

test_data_features = vectorizer.transform(clean_test_data)

In [30]:
# convert to array

train_data_features = train_data_features.toarray()

In [31]:
# check shapes

train_data_features.shape, test_data_features.shape

((1182, 5000), (395, 5000))

In [32]:
# I wanted check that the features corpus was as expected - removed print statement for readability

vocab = vectorizer.get_feature_names()

In [33]:
vocab[0:200]

['000',
 '000 people',
 '000 receiving',
 '000 receiving yards',
 '000 rush',
 '06',
 '07',
 '07 13',
 '07 13 20',
 '10',
 '10 16',
 '10 asts rebs',
 '10 days',
 '10 hof',
 '10 hof players',
 '10 iq',
 '10 iq points',
 '10 offensive',
 '10 offensive lines',
 '10 year',
 '10 year extension',
 '10 years',
 '100',
 '100 players',
 '100 players 2019',
 '11',
 '12',
 '12 17 2013',
 '12 games',
 '12 games 1987',
 '125m',
 '13',
 '13 20',
 '14',
 '14 000',
 '14 000 seats',
 '14 carries',
 '14 days',
 '15',
 '15 tagged',
 '16',
 '17',
 '170',
 '170 wins',
 '170 wins went',
 '18',
 '18 point',
 '18 point comeback',
 '19',
 '19 protocols',
 '19 protocols 2020',
 '19 tests',
 '19 years',
 '1937',
 '1937 packers',
 '1967',
 '1967 86',
 '1967 86 oakland',
 '1970',
 '1970 steelers',
 '1970 steelers retained',
 '198',
 '198 2m',
 '198 2m 2021',
 '1985',
 '1987',
 '1988',
 '1989',
 '1991',
 '1992',
 '1993',
 '1995',
 '1995 game',
 '1996',
 '1998',
 '1999',
 '1m',
 '1st',
 '1st quarter',
 '1st quarter 

## MODELING

### Logistic Regression

In [34]:
from sklearn.linear_model import LogisticRegression

In [35]:
# fit logistic regression model

lr = LogisticRegression(penalty='l2')

In [36]:
# shape check

train_data_features.shape, y_train.shape

((1182, 5000), (1182,))

In [37]:
lr.fit(train_data_features, y_train)

LogisticRegression()

In [38]:
lr.score(train_data_features, y_train)

0.9949238578680203

In [39]:
lr.score(test_data_features, y_test)

0.9113924050632911

### Feature comparison

Creates a dataframe that matches features to coefficients

In [40]:
coef_list = lr.coef_.tolist()

In [41]:
coef_list = coef_list[0]

In [42]:
coef_df = pd.DataFrame({'features': vectorizer.get_feature_names(),
                        'coefs': coef_list})

In [43]:
coef_df.sort_values(by = ['coefs'])

,features,coefs
2857,nba,-2.582027
768,bubble,-2.089666
2474,lebron,-1.624766
570,basketball,-1.282128
3105,orlando,-1.102807
...,...,...
3642,redskins,1.116065
2622,mahomes,1.161758
3508,qb,1.564634
2053,highlight,1.698194


### Let's throw out these unfair words and rerun

In [44]:
stopwords = set(stopwords.words('english'))

extra_stopwords = ['nba', 'basketball', 'football', 'nfl']

stopwords.update(extra_stopwords)

In [45]:
vectorizer = CountVectorizer(analyzer = "word",
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = stopwords,
                             max_features = 5000,
                             ngram_range = (1, 3))

train_data_features = vectorizer.fit_transform(clean_train_data)

test_data_features = vectorizer.transform(clean_test_data)

train_data_features = train_data_features.toarray()

train_data_features.shape, test_data_features.shape

((1182, 5000), (395, 5000))

In [46]:
lr.fit(train_data_features, y_train)

LogisticRegression()

In [47]:
lr.score(train_data_features, y_train)

0.9923857868020305

In [48]:
lr.score(test_data_features, y_test)

0.8734177215189873

In [49]:
coef_list = lr.coef_.tolist()
coef_list = coef_list[0]

coef_df = pd.DataFrame({'features' : vectorizer.get_feature_names(),
                       'coefs' : coef_list})

coef_df.sort_values(by = ['coefs'])

,features,coefs
807,bubble,-2.268813
2508,lebron,-1.667516
3349,playoffs,-1.193752
3076,orlando,-1.151388
1372,disney,-1.046233
...,...,...
3644,redskins,1.220122
2650,mahomes,1.224853
136,2020,1.465090
3496,qb,1.554753


### Decision Tree

In [50]:
from sklearn.tree import DecisionTreeClassifier

In [51]:
tree = DecisionTreeClassifier()

In [52]:
tree.fit(train_data_features, y_train)

DecisionTreeClassifier()

In [53]:
tree.score(train_data_features, y_train)

0.9991539763113367

In [54]:
tree.score(test_data_features, y_test)

0.7924050632911392

### Random Forest

In [55]:
from sklearn.ensemble import RandomForestClassifier

In [56]:
forest = RandomForestClassifier(n_estimators = 100)

In [57]:
forest.fit(train_data_features, y_train)

RandomForestClassifier()

In [58]:
forest.score(train_data_features, y_train)

0.9991539763113367

In [59]:
forest.score(test_data_features, y_test)

0.8227848101265823

###  Matrix on Logistic Regression

In [60]:
from sklearn.metrics import confusion_matrix

In [61]:
y_pred = lr.predict(test_data_features)

In [62]:
cm = confusion_matrix(y_test, y_pred)

In [63]:
cm_df = pd.DataFrame(cm,
                    columns=['predict_neg', 'predict_pos'],
                    index = ['actual_neg', 'actual_pos'])

In [64]:
cm_df

,predict_neg,predict_pos
actual_neg,159,18
actual_pos,32,186


## Checking where our model failed

In [65]:
comparison_df = pd.DataFrame({'y_actual' : y_test,
             'y_predicted' : y_pred})

In [66]:
mismatch_df = comparison_df[comparison_df['y_actual'] != comparison_df['y_predicted']]

In [67]:
mismatch2_df = pd.concat([mismatch_df, X_test], axis = 1)

In [68]:
# All incorrect predictions with titles

mismatches = mismatch2_df.dropna()

In [69]:
mismatches

,y_actual,y_predicted,title
t3_hoh1b0,nfl,nba,marcus hayes if intolerance cannot be tolerated when directed at black people it cannot be tolerated when it comes from black people
t3_hqgmgc,nfl,nba,andrew thomas switching from left to right tackle natural after a few reps
t3_horz0w,nfl,nba,charles robinson hearing some names of nfl players with serious asthma concerns who may opt out of the season including one elite level guy haven t heard any final decisions yet but it s pretty clear some players are seriously weighing sitting out 2020
t3_hrl775,nfl,nba,kareem abdul jabbar where is the outrage over anti semitism in sports and hollywood
t3_hrhzb7,nfl,nba,hembekides from 2001 06 peyton manning started 40 games in which his team allowed fewer than 20 points he went 40 0
t3_hpmq34,nba,nfl,youngmisuk jamal murray i m not in game shape if someone s in game shape here man they re doing something they shouldn t be doing
t3_hof5wh,nfl,nba,who is the one player from your divisional rivals you always feared or respected
t3_hr690f,nba,nfl,social worker asking nba fans
t3_hre5px,nba,nfl,if you could go back to june 2010 and tell your teams owner 5 words knowing what would happen in the next decade what would you say
t3_hnmsz2,nfl,nba,mcmann covid wasn t caused by football or caused by sports jim harbaugh said there s no agreeing expert view right now that i m aware of that sports is going to make it worse it s part of our society we re going to have to deal with it


### Let's try TF-IDF

Term Frequency / Inverse Document Frequency

TF(w) = (Number of times term w appears in a document) / (Total number of terms in the document)

IDF(w) = log_e(Total number of documents / Number of documents with term w in it)

In [70]:
tfidf_vec = TfidfVectorizer(analyzer="word",
                            tokenizer=None,
                            preprocessor=None,
                            stop_words=['nba', 'nfl', 'football', 'basketball'],
                            max_features=5000,
                            ngram_range=(1, 3))

In [71]:
train_data_features = tfidf_vec.fit_transform(clean_train_data)

test_data_features = tfidf_vec.transform(clean_test_data)

train_data_features = train_data_features.toarray()

train_data_features.shape, test_data_features.shape

((1182, 5000), (395, 5000))

In [72]:
lr.fit(train_data_features, y_train)

LogisticRegression()

In [73]:
lr.score(train_data_features, y_train)

0.9729272419627749

In [74]:
lr.score(test_data_features, y_test)

0.8481012658227848

### Let's try on some other subreddits

In [75]:
train = pd.concat([politics_test, conservative_test])

In [76]:
X = train[['title']]
y = train['subreddit']

In [77]:
# politics_test = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/politics.json')
# conservative_test = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/conservative.json')

In [78]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify = y)

In [79]:
politics_test = politics_test.drop(columns='selftext')
conservative_test = conservative_test.drop(columns='selftext')

train = pd.concat([politics_test, conservative_test])
tokenizer = RegexpTokenizer(r'\w+')

train['title'] = train['title'].map(lambda x: tokenizer.tokenize(x.lower()))
train['title'] = train['title'].map(lambda x: ' '.join(x))

In [80]:
# create our training data list - this is a list of strings, with each string being a post title

clean_train_data = []

for traindata in X_train['title']:
    clean_train_data.append(traindata)
    
    
# create test data list

clean_test_data = []

for testdata in X_test['title']:
    clean_test_data.append(testdata)

In [81]:
vectorizer = CountVectorizer(analyzer="word",
                             tokenizer=None,
                             preprocessor=None,
                             stop_words='english',
                             max_features=5000,
                             ngram_range=(1, 3))

train_data_features = vectorizer.fit_transform(clean_train_data)

test_data_features = vectorizer.transform(clean_test_data)

train_data_features = train_data_features.toarray()

train_data_features.shape, test_data_features.shape

vocab = vectorizer.get_feature_names()

### Modeling

In [82]:
lr = LogisticRegression(penalty = 'l2')

In [83]:
train_data_features.shape, y_train.shape

((1390, 5000), (1390,))

In [84]:
lr.fit(train_data_features, y_train)

lr.score(train_data_features, y_train)

0.9776978417266187

In [85]:
lr.score(test_data_features, y_test)

0.7586206896551724

In [86]:
coef_list = lr.coef_.tolist()

coef_list = coef_list[0]

In [87]:
coef_df = pd.DataFrame({'features' : vectorizer.get_feature_names(),
                       'coefs' : coef_list})

coef_df.sort_values(by = ['coefs'])

,features,coefs
2634,media,-1.494181
2482,loses,-1.253657
562,blm,-1.172104
3173,point,-1.144246
2893,nyc,-0.996739
...,...,...
2279,kids,1.120645
2163,ivanka,1.168081
1631,finally,1.218578
1572,fauci,1.361728


# Understanding word vectors

... for, like, actual poets. By [Allison Parrish](http://www.decontextualize.com/)


## Why word vectors?

Poetry is, at its core, the art of identifying and manipulating linguistic similarity. I have discovered a truly marvelous proof of this, which this notebook is too narrow to contain. (By which I mean: I will elaborate on this some other time)

## Animal similarity and simple linear algebra

We'll begin by considering a small subset of English: words for animals. Our task is to be able to write computer programs to find similarities among these words and the creatures they designate. To do this, we might start by making a spreadsheet of some animals and their characteristics. For example:

![Animal spreadsheet](http://static.decontextualize.com/snaps/animal-spreadsheet.png)

This spreadsheet associates a handful of animals with two numbers: their cuteness and their size, both in a range from zero to one hundred. (The values themselves are simply based on my own judgment. Your taste in cuteness and evaluation of size may differ significantly from mine. As with all data, these data are simply a mirror reflection of the person who collected them.)

These values give us everything we need to make determinations about which animals are similar (at least, similar in the properties that we've included in the data). Try to answer the following question: Which animal is most similar to a capybara? You could go through the values one by one and do the math to make that evaluation, but visualizing the data as points in 2-dimensional space makes finding the answer very intuitive:

![Animal space](http://static.decontextualize.com/snaps/animal-space.png)

The plot shows us that the closest animal to the capybara is the panda bear (again, in terms of its subjective size and cuteness). One way of calculating how "far apart" two points are is to find their *Euclidean distance*. (This is simply the length of the line that connects the two points.) For points in two dimensions, Euclidean distance can be calculated with the following Python function:

In [88]:
import numpy as np
def distance2d(x1, y1, x2, y2):
    return np.linalg.norm(np.array([x1, y1])-np.array([x2, y2]))

So, the distance between "capybara" (70, 30) and "panda" (74, 40):

In [89]:
distance2d(70, 30, 75, 40) # panda and capybara

11.180339887498949

... is less than the distance between "tarantula" and "elephant":

In [90]:
distance2d(8, 3, 65, 90) # tarantula and elephant

104.0096149401583

Modeling animals in this way has a few other interesting properties. For example, you can pick an arbitrary point in "animal space" and then find the animal closest to that point. If you imagine an animal of size 25 and cuteness 30, you can easily look at the space to find the animal that most closely fits that description: the chicken.

Reasoning visually, you can also answer questions like: what's halfway between a chicken and an elephant? Simply draw a line from "elephant" to "chicken," mark off the midpoint and find the closest animal. (According to our chart, halfway between an elephant and a chicken is a horse.)

You can also ask: what's the *difference* between a hamster and a tarantula? According to our plot, it's about seventy five units of cute (and a few units of size).

The relationship of "difference" is an interesting one, because it allows us to reason about *analogous* relationships. In the chart below, I've drawn an arrow from "tarantula" to "hamster" (in blue):

![Animal analogy](http://static.decontextualize.com/snaps/animal-space-analogy.png)

You can understand this arrow as being the *relationship* between a tarantula and a hamster, in terms of their size and cuteness (i.e., hamsters and tarantulas are about the same size, but hamsters are much cuter). In the same diagram, I've also transposed this same arrow (this time in red) so that its origin point is "chicken." The arrow ends closest to "kitten." What we've discovered is that the animal that is about the same size as a chicken but much cuter is... a kitten. To put it in terms of an analogy:

    Tarantulas are to hamsters as chickens are to kittens.
    
A sequence of numbers used to identify a point is called a *vector*, and the kind of math we've been doing so far is called *linear algebra.* (Linear algebra is surprisingly useful across many domains: It's the same kind of math you might do to, e.g., simulate the velocity and acceleration of a sprite in a video game.)

A set of vectors that are all part of the same data set is often called a *vector space*. The vector space of animals in this section has two *dimensions*, by which I mean that each vector in the space has two numbers associated with it (i.e., two columns in the spreadsheet). The fact that this space has two dimensions just happens to make it easy to *visualize* the space by drawing a 2D plot. But most vector spaces you'll work with will have more than two dimensions—sometimes many hundreds. In those cases, it's more difficult to visualize the "space," but the math works pretty much the same.

## Language with vectors: colors

So far, so good. We have a system in place—albeit highly subjective—for talking about animals and the words used to name them. I want to talk about another vector space that has to do with language: the vector space of colors.

Colors are often represented in computers as vectors with three dimensions: red, green, and blue. Just as with the animals in the previous section, we can use these vectors to answer questions like: which colors are similar? What's the most likely color name for an arbitrarily chosen set of values for red, green and blue? Given the names of two colors, what's the name of those colors' "average"?

We'll be working with this [color data](https://github.com/dariusk/corpora/blob/master/data/colors/xkcd.json) from the [xkcd color survey](https://blog.xkcd.com/2010/05/03/color-survey-results/). The data relates a color name to the RGB value associated with that color. [Here's a page that shows what the colors look like](https://xkcd.com/color/rgb/). Download the color data and put it in the same directory as this notebook.

A few notes before we proceed:

* The linear algebra functions implemented below (`addv`, `meanv`, etc.) are slow, potentially inaccurate, and shouldn't be used for "real" code—I wrote them so beginner programmers can understand how these kinds of functions work behind the scenes. Use [numpy](http://www.numpy.org/) for fast and accurate math in Python.
* If you're interested in perceptually accurate color math in Python, consider using the [colormath library](http://python-colormath.readthedocs.io/en/latest/).

Now, import the `json` library and load the color data:

In [91]:
import requests

In [92]:
resp = requests.get('https://raw.githubusercontent.com/dariusk/corpora/master/data/colors/xkcd.json')
color_data = resp.json()

The following function converts colors from hex format (`#1a2b3c`) to a tuple of integers:

In [93]:
def hex_to_int(s):
    return int(s[1:3], 16), int(s[3:5], 16), int(s[5:6], 16)

And the following cell creates a dictionary and populates it with mappings from color names to RGB vectors for each color in the data:

In [94]:
colors = dict()
for item in color_data['colors']:
    colors[item["color"]] = hex_to_int(item["hex"])

Testing it out:

In [95]:
colors['olive']

(110, 117, 0)

In [96]:
colors['red']

(229, 0, 0)

In [97]:
colors['black']

(0, 0, 0)

In [98]:
colors.keys()

dict_keys(['cloudy blue', 'dark pastel green', 'dust', 'electric lime', 'fresh green', 'light eggplant', 'nasty green', 'really light blue', 'tea', 'warm purple', 'yellowish tan', 'cement', 'dark grass green', 'dusty teal', 'grey teal', 'macaroni and cheese', 'pinkish tan', 'spruce', 'strong blue', 'toxic green', 'windows blue', 'blue blue', 'blue with a hint of purple', 'booger', 'bright sea green', 'dark green blue', 'deep turquoise', 'green teal', 'strong pink', 'bland', 'deep aqua', 'lavender pink', 'light moss green', 'light seafoam green', 'olive yellow', 'pig pink', 'deep lilac', 'desert', 'dusty lavender', 'purpley grey', 'purply', 'candy pink', 'light pastel green', 'boring green', 'kiwi green', 'light grey green', 'orange pink', 'tea green', 'very light brown', 'egg shell', 'eggplant purple', 'powder pink', 'reddish grey', 'baby shit brown', 'liliac', 'stormy blue', 'ugly brown', 'custard', 'darkish pink', 'deep brown', 'greenish beige', 'manilla', 'off blue', 'battleship gre

### Vector math

Before we keep going, we'll need some functions for performing basic vector arithmetic. These functions will work with vectors in spaces of any number of dimensions.

The first function returns the Euclidean distance between two points:

In [99]:
import math
def distance(coord1, coord2):
    return np.linalg.norm(np.array(coord1)-np.array(coord2))
distance([10, 1], [5, 2])

5.0990195135927845

The `subtractv` function subtracts one vector from another:

In [100]:
def subtractv(coord1, coord2):
    return list(np.array(coord1) - np.array(coord2))
subtractv([10, 1], [5, 2])

[5, -1]

The `addv` vector adds two vectors together:

In [101]:
def addv(coord1, coord2):
    return list(np.array(coord1) + np.array(coord2))
addv([10, 1], [5, 2])

[15, 3]

And the `meanv` function takes a list of vectors and finds their mean or average:

In [102]:
def meanv(coords):
    return list(np.mean(coords, axis=0))
meanv([[0, 1], [2, 2], [4, 3]])

[2.0, 2.0]

Just as a test, the following cell shows that the distance from "red" to "green" is greater than the distance from "red" to "pink":

In [103]:
distance(colors['red'], colors['green']) > distance(colors['red'], colors['pink'])

True

### Finding the closest item

Just as we wanted to find the animal that most closely matched an arbitrary point in cuteness/size space, we'll want to find the closest color name to an arbitrary point in RGB space. The easiest way to find the closest item to an arbitrary vector is simply to find the distance between the target vector and each item in the space, in turn, then sort the list from closest to farthest. The `closest()` function below does just that. By default, it returns a list of the ten closest items to the given vector.

> Note: Calculating "closest neighbors" like this is fine for the examples in this notebook, but unmanageably slow for vector spaces of any appreciable size. As your vector space grows, you'll want to move to a faster solution, like SciPy's [kdtree](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.spatial.KDTree.html) or [Annoy](https://pypi.python.org/pypi/annoy).

In [104]:
def closest(space, coord, n=10):
    return sorted(space.keys(), key=lambda x: distance(coord, space[x]))[:n]

Testing it out, we can find the ten colors closest to "red":

In [105]:
closest(colors, colors['white'])

['white',
 'off white',
 'eggshell',
 'pale grey',
 'ivory',
 'cream',
 'ecru',
 'creme',
 'light beige',
 'egg shell']

... or the ten colors closest to (150, 60, 150):

In [106]:
closest(colors, [255, 60, 150])

['violet pink',
 'light magenta',
 'barbie pink',
 'pink/purple',
 'purple pink',
 'candy pink',
 'heliotrope',
 'pink purple',
 'purpleish pink',
 'pinkish purple']

### Color magic

The magical part of representing words as vectors is that the vector operations we defined earlier appear to operate on language the same way they operate on numbers. For example, if we find the word closest to the vector resulting from subtracting "red" from "purple," we get a series of "blue" colors:

In [107]:
closest(colors, subtractv(colors['green'], colors['purple']))

['kelley green',
 'irish green',
 'emerald green',
 'emerald',
 'teal blue',
 'teal',
 'bluegreen',
 'ocean',
 'kelly green',
 'jungle green']

This matches our intuition about RGB colors, which is that purple is a combination of red and blue. Take away the red, and blue is all you have left.

You can do something similar with addition. What's blue plus green?

In [108]:
closest(colors, addv(colors['blue'], colors['green']))

['spearmint',
 'aqua',
 'wintergreen',
 'bright turquoise',
 'bright aqua',
 'bright light blue',
 'minty green',
 'highlighter green',
 'electric green',
 'aqua green']

That's right, it's something like turquoise or cyan! What if we find the average of black and white? Predictably, we get gray:

In [109]:
# the average of black and white: medium grey
closest(colors, meanv([colors['black'], colors['white']]))

['medium grey',
 'drab',
 'brownish grey',
 'shit green',
 'steel',
 'brown grey',
 'swamp green',
 'khaki green',
 'grey brown',
 'camo']

Just as with the tarantula/hamster example from the previous section, we can use color vectors to reason about relationships between colors. In the cell below, finding the difference between "pink" and "red" then adding it to "blue" seems to give us a list of colors that are to blue what pink is to red (i.e., a slightly lighter, less saturated shade):

In [110]:
# an analogy: pink is to red as X is to blue
pink_to_red = subtractv(colors['orange'], colors['red'])
closest(colors, addv(pink_to_red, colors['blue']))

['topaz',
 'dark seafoam',
 'green teal',
 'green',
 'tealish',
 'dark mint green',
 'turquoise blue',
 'algae green',
 'green blue',
 'jade']

Another example of color analogies: Navy is to blue as true green/dark grass green is to green:

In [111]:
# another example: 
navy_to_blue = subtractv(colors['navy'], colors['blue'])
closest(colors, addv(navy_to_blue, colors['green']))

['dark sea green',
 'blue green',
 'jungle green',
 'dark cyan',
 'greenish blue',
 'nice blue',
 'deep aqua',
 'water blue',
 'true green',
 'emerald green']

The examples above are fairly simple from a mathematical perspective but nevertheless *feel* magical: they're demonstrating that it's possible to use math to reason about how people use language.

### Interlude: A Love Poem That Loses Its Way

In [112]:
import random
red = colors['red']
blue = colors['blue']
for i in range(14):
    rednames = closest(colors, red)
    bluenames = closest(colors, blue)
    print(f"Roses are {rednames[0]}, violets are {bluenames[0]}")
    red = colors[random.choice(rednames[1:])]
    blue = colors[random.choice(bluenames[1:])]

Roses are red, violets are blue
Roses are cherry, violets are darkish blue
Roses are cerise, violets are prussian blue
Roses are fuchsia, violets are evergreen
Roses are red, violets are bottle green
Roses are dark hot pink, violets are british racing green
Roses are lipstick, violets are evergreen
Roses are dark hot pink, violets are prussian blue
Roses are red, violets are darkgreen
Roses are cerise, violets are racing green
Roses are pinkish red, violets are darkgreen
Roses are fuchsia, violets are prussian blue
Roses are hot magenta, violets are racing green
Roses are hot pink, violets are forest green


### Doing bad digital humanities with color vectors

With the tools above in hand, we can start using our vectorized knowledge of language toward academic ends. In the following example, I'm going to calculate the average color of Bram Stoker's *Dracula*.

First, we'll load [spaCy](https://spacy.io/):

In [113]:
import spacy
nlp = spacy.load('en_core_web_sm')

To calculate the average color, we'll follow these steps:

1. Parse the text into words
2. Check every word to see if it names a color in our vector space. If it does, add it to a list of vectors.
3. Find the average of that list of vectors.
4. Find the color(s) closest to that average vector.

The following cell performs steps 1-3:

In [114]:
resp = requests.get('http://www.gutenberg.org/cache/epub/345/pg345.txt')
dracula = nlp(resp.text)
# use word.lower_ to normalize case
drac_colors = [colors[word.lower_] for word in dracula if word.lower_ in colors]
avg_color = meanv(drac_colors)
print(avg_color)

[147.44839067702551, 113.65371809100999, 5.871254162042176]


Now, we'll pass the averaged color vector to the `closest()` function, yielding... well, it's just a brown mush, which is kinda what you'd expect from adding a bunch of colors together willy-nilly.

In [115]:
closest(colors, avg_color)

['faded purple',
 'reddish grey',
 'deep lilac',
 'poo',
 'puke brown',
 'hazel',
 'brownish',
 'dirt',
 'mocha',
 'dark lilac']

On the other hand, here's what we get when we average the colors of Charlotte Perkins Gilman's classic *The Yellow Wallpaper*. The result definitely reflects the content of the story, so maybe we're on to something here.

In [116]:
resp = requests.get('http://www.gutenberg.org/cache/epub/1952/pg1952.txt')
yellow = nlp(resp.text)
wallpaper_colors = [colors[word.lower_] for word in yellow if word.lower_ in colors]
avg_color = meanv(wallpaper_colors)
closest(colors, avg_color)

['dirty yellow',
 'vomit yellow',
 'ugly yellow',
 'puke yellow',
 'mustard yellow',
 'silver',
 'greenish beige',
 'light periwinkle',
 'greenish tan',
 'olive yellow']

Exercise for the reader: Use the vector arithmetic functions to rewrite a text, making it...

* more blue (i.e., add `colors['blue']` to each occurrence of a color word); or
* more light (i.e., add `colors['white']` to each occurrence of a color word); or
* darker (i.e., attenuate each color. You might need to write a vector multiplication function to do this one right.)

## Distributional semantics

In the previous section, the examples are interesting because of a simple fact: colors that we think of as similar are "closer" to each other in RGB vector space. In our color vector space, or in our animal cuteness/size space, you can think of the words identified by vectors close to each other as being *synonyms*, in a sense: they sort of "mean" the same thing. They're also, for many purposes, *functionally identical*. Think of this in terms of writing, say, a search engine. If someone searches for "mauve trousers," then it's probably also okay to show them results for, say,

In [117]:
for cname in closest(colors, colors['mauve']):
    print(cname, "trousers")

mauve trousers
pinkish brown trousers
leather trousers
clay brown trousers
caramel trousers
soft purple trousers
light urple trousers
clay trousers
golden brown trousers
bronze trousers


That's all well and good for color words, which intuitively seem to exist in a multidimensional continuum of perception, and for our animal space, where we've written out the vectors ahead of time. But what about... arbitrary words? Is it possible to create a vector space for all English words that has this same "closer in space is closer in meaning" property?

To answer that, we have to back up a bit and ask the question: what does *meaning* mean? No one really knows, but one theory popular among computational linguists, computer scientists and other people who make search engines is the [Distributional Hypothesis](https://en.wikipedia.org/wiki/Distributional_semantics), which states that:

    Linguistic items with similar distributions have similar meanings.
    
What's meant by "similar distributions" is *similar contexts*. Take for example the following sentences:

    It was really cold yesterday.
    It will be really warm today, though.
    It'll be really hot tomorrow!
    Will it be really cool Tuesday?
    
According to the Distributional Hypothesis, the words `cold`, `warm`, `hot` and `cool` must be related in some way (i.e., be close in meaning) because they occur in a similar context, i.e., between the word "really" and a word indicating a particular day. (Likewise, the words `yesterday`, `today`, `tomorrow` and `Tuesday` must be related, since they occur in the context of a word indicating a temperature.)

In other words, according to the Distributional Hypothesis, a word's meaning is just a big list of all the contexts it occurs in. Two words are closer in meaning if they share contexts.

## Word vectors by counting contexts

So how do we turn this insight from the Distributional Hypothesis into a system for creating general-purpose vectors that capture the meaning of words? Maybe you can see where I'm going with this. What if we made a *really big* spreadsheet that had one column for every context for every word in a given source text. Let's use a small source text to begin with, such as this excerpt from Dickens:

    It was the best of times, it was the worst of times.

Such a spreadsheet might look something like this:

![dickens contexts](http://static.decontextualize.com/snaps/best-of-times.png)

The spreadsheet has one column for every possible context, and one row for every word. The values in each cell correspond with how many times the word occurs in the given context. The numbers in the columns constitute that word's vector, i.e., the vector for the word `of` is

    [0, 0, 0, 0, 1, 0, 0, 0, 1, 0]
    
Because there are ten possible contexts, this is a ten dimensional space! It might be strange to think of it, but you can do vector arithmetic on vectors with ten dimensions just as easily as you can on vectors with two or three dimensions, and you could use the same distance formula that we defined earlier to get useful information about which vectors in this space are similar to each other. In particular, the vectors for `best` and `worst` are actually the same (a distance of zero), since they occur only in the same context (`the ___ of`):

    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
    
Of course, the conventional way of thinking about "best" and "worst" is that they're *antonyms*, not *synonyms*. But they're also clearly two words of the same kind, with related meanings (through opposition), a fact that is captured by this distributional model.

### Contexts and dimensionality

Of course, in a corpus of any reasonable size, there will be many thousands if not many millions of possible contexts. It's difficult enough working with a vector space of ten dimensions, let alone a vector space of a million dimensions! It turns out, though, that many of the dimensions end up being superfluous and can either be eliminated or combined with other dimensions without significantly affecting the predictive power of the resulting vectors. The process of getting rid of superfluous dimensions in a vector space is called [dimensionality reduction](https://en.wikipedia.org/wiki/Dimensionality_reduction), and most implementations of count-based word vectors make use of dimensionality reduction so that the resulting vector space has a reasonable number of dimensions (say, 100—300, depending on the corpus and application).

The question of how to identify a "context" is itself very difficult to answer. In the toy example above, we've said that a "context" is just the word that precedes and the word that follows. Depending on your implementation of this procedure, though, you might want a context with a bigger "window" (e.g., two words before and after), or a non-contiguous window (skip a word before and after the given word). You might exclude certain "function" words like "the" and "of" when determining a word's context, or you might [lemmatize](https://en.wikipedia.org/wiki/Lemmatisation) the words before you begin your analysis, so two occurrences with different "forms" of the same word count as the same context. These are all questions open to research and debate, and different implementations of procedures for creating count-based word vectors make different decisions on this issue.

### GloVe vectors

But you don't have to create your own word vectors from scratch! Many researchers have made downloadable databases of pre-trained vectors. One such project is Stanford's [Global Vectors for Word Representation (GloVe)](https://nlp.stanford.edu/projects/glove/). These 300-dimensional vectors are included with spaCy, and they're the vectors we'll be using for the rest of this tutorial.

## Word vectors in spaCy

Okay, let's have some fun with real word vectors. We're going to use the GloVe vectors that come with spaCy to creatively analyze and manipulate the text of Bram Stoker's *Dracula*. First, make sure you've got `spacy` imported:

In [118]:
dracula[560]

hour

In [119]:
_.vector

array([ 9.5816952e-01,  2.9433322e+00,  3.2474852e+00,  9.4183958e-01,
        9.0796781e-01, -1.4559157e+00, -1.0931147e+00,  2.2580385e-02,
       -9.4498986e-01,  1.8858352e+00,  1.7374688e-01, -1.9966578e+00,
        1.6485956e+00,  4.1239840e-01,  2.3332387e-01,  9.1011721e-01,
        4.3031251e-01, -6.4356017e-01, -1.3078554e+00,  3.6566243e+00,
       -5.3426570e-01, -1.3150523e+00, -1.7053117e+00, -9.8732960e-01,
        1.6114790e+00,  2.3365812e+00, -1.5034729e+00, -1.3236241e+00,
        1.3935536e-01, -1.3248553e+00, -2.7768307e+00,  7.5319695e-01,
        4.9489999e-01, -7.1741229e-01,  7.3097110e-01, -1.8518646e+00,
       -6.6337895e-01, -1.4353507e+00,  1.1697152e+00, -4.1833043e-01,
        4.7056117e+00,  2.8764541e+00, -2.1899223e+00, -3.0172282e-01,
        8.0876392e-01, -5.6309414e-01,  4.9557450e-01, -3.2273710e-01,
       -2.7931430e+00, -1.3081805e-01, -1.8106464e+00, -3.3482976e+00,
       -3.5771877e-01, -1.4838344e+00, -1.7561685e+00,  3.2123671e+00,
      

In [123]:
# previously we've used the _sm model, which doesn't include all vectors.
# !pip3 install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.0/en_core_web_lg-2.2.0.tar.gz
import en_core_web_lg
nlp = en_core_web_lg.load()

In [124]:
resp = requests.get('http://www.gutenberg.org/cache/epub/345/pg345.txt')
dracula = nlp(resp.text)


And the cell below creates a list of unique words (or tokens) in the text, as a list of strings.

In [125]:
# all of the words in the text file
tokens = list(set([w.text for w in dracula if w.is_alpha]))

You can see the vector of any word in spaCy's vocabulary using the `vocab` attribute, like so:

In [126]:
nlp.vocab['alligator'].vector

array([-4.9304e-01,  1.2459e-01,  2.6142e-01,  3.8393e-02,  1.4018e-01,
        5.7285e-01, -5.0449e-01,  1.5094e-01,  7.3356e-02,  2.3308e-01,
       -1.6048e-01, -5.1184e-01, -1.4028e-01, -3.0110e-01, -4.3446e-01,
       -2.8257e-02,  1.9100e-03,  6.5157e-01,  1.4855e-01, -3.7255e-01,
       -4.6619e-01, -1.7223e-01, -5.1794e-01, -2.5453e-01, -2.6785e-01,
       -6.7776e-02, -3.0085e-01, -3.3212e-01, -1.5862e-01, -1.2336e-01,
       -3.8935e-01, -3.5551e-01, -3.6182e-01,  2.3197e-02, -1.7486e-01,
        2.2345e-01,  6.7557e-01,  9.6939e-03, -9.5640e-02,  1.2073e-01,
       -1.2016e-01, -1.6161e-01,  1.0661e-01, -4.5233e-01,  4.5991e-01,
        1.0367e-02, -1.0842e-01, -3.6163e-01,  2.5105e-01,  9.5780e-03,
       -6.1363e-01,  2.7494e-01,  2.1783e-01, -1.0663e-01, -9.5146e-03,
        2.7570e-01,  2.5888e-02, -5.6749e-02,  6.7667e-03,  5.3735e-01,
       -1.2224e-01,  8.8448e-02,  4.6094e-01,  4.6881e-02, -6.6557e-01,
       -3.9493e-01,  2.7362e-01,  1.2302e-01,  2.8594e-01, -2.52

For the sake of convenience, the following function gets the vector of a given string from spaCy's vocabulary:

In [127]:
def vec(s):
    return nlp.vocab[s].vector

### Cosine similarity and finding closest neighbors

In [128]:
from sklearn.metrics.pairwise import cosine_similarity

The following cell shows that the cosine similarity between `dog` and `puppy` is larger than the similarity between `trousers` and `octopus`, thereby demonstrating that the vectors are working how we expect them to:

In [129]:
(cosine_similarity(vec('dog').reshape(1, -1), vec('puppy').reshape(1, -1))[0][0] > 
    cosine_similarity(vec('trousers').reshape(1, -1), vec('octopus').reshape(1, -1))[0][0])

True

The following cell defines a function that iterates through a list of tokens and returns the token whose vector is most similar to a given vector.

In [130]:
def spacy_closest(token_list, vec_to_check, n=10):
    return sorted(token_list,
                  key=lambda x: cosine_similarity(vec_to_check.reshape(1, -1), 
                                                  vec(x).reshape(1, -1))[0][0],
                  reverse=True)[:n]

Using this function, we can get a list of synonyms, or words closest in meaning (or distribution, depending on how you look at it), to any arbitrary word in spaCy's vocabulary. In the following example, we're finding the words in *Dracula* closest to "basketball":

In [131]:
# what's the closest equivalent of basketball?
spacy_closest(tokens, vec("basketball"))

['tennis',
 'coach',
 'game',
 'teams',
 'Junior',
 'junior',
 'Team',
 'school',
 'boys',
 'leagues']

### Fun with spaCy, Dracula, and vector arithmetic

Now we can start doing vector arithmetic and finding the closest words to the resulting vectors. For example, what word is closest to the halfway point between day and night?

In [132]:
# halfway between day and night
spacy_closest(tokens, np.array(meanv([vec("day"), vec("night")])))

['night',
 'day',
 'Day',
 'evening',
 'Evening',
 'Morning',
 'morning',
 'afternoon',
 'nights',
 'Nights']

Variations of `night` and `day` are still closest, but after that we get words like `evening` and `morning`, which are indeed halfway between day and night!

Here are the closest words in _Dracula_ to "wine":

In [133]:
spacy_closest(tokens, vec("wine"))

['wine',
 'beer',
 'bottle',
 'Drink',
 'drink',
 'cellar',
 'fruit',
 'bottles',
 'brandy',
 'taste']

If you subtract "alcohol" from "wine" and find the closest words to the resulting vector, you're left with simply a lovely dinner:

In [134]:
spacy_closest(tokens, np.array(subtractv(vec("wine"), vec("alcohol"))))

['wine',
 'cellar',
 'exquisite',
 'fabulous',
 'splendid',
 'magnificent',
 'delightful',
 'Dinner',
 'dinner',
 'sparkling']

The closest words to "water":

In [135]:
spacy_closest(tokens, vec("water"))

['water',
 'waters',
 'Salt',
 'salt',
 'pond',
 'dry',
 'liquid',
 'ocean',
 'boiling',
 'heat']

But if you add "frozen" to "water," you get "ice":

In [136]:
spacy_closest(tokens, np.array(addv(vec("water"), vec("frozen"))))

['water',
 'cold',
 'ice',
 'Salt',
 'salt',
 'dry',
 'fresh',
 'liquid',
 'boiling',
 'milk']

You can even do analogies! For example, the words most similar to "grass":

In [137]:
spacy_closest(tokens, vec("grass"))

['grass',
 'lawn',
 'trees',
 'greens',
 'grassy',
 'garden',
 'GARDEN',
 'sand',
 'foliage',
 'tree']

If you take the difference of "blue" and "sky" and add it to grass, you get the analogous word ("green"):

In [138]:
# analogy: blue is to sky as X is to grass
blue_to_sky = subtractv(vec("blue"), vec("sky"))
spacy_closest(tokens, np.array(addv(blue_to_sky, vec("grass"))))

['grass',
 'GREEN',
 'green',
 'Green',
 'yellow',
 'red',
 'Red',
 'purple',
 'lawn',
 'pink']

## Further resources

* [Word2vec](https://en.wikipedia.org/wiki/Word2vec) is another procedure for producing word vectors which uses a predictive approach rather than a context-counting approach. [This paper](http://clic.cimec.unitn.it/marco/publications/acl2014/baroni-etal-countpredict-acl2014.pdf) compares and contrasts the two approaches. (Spoiler: it's kind of a wash.)
* If you want to train your own word vectors on a particular corpus, the popular Python library [gensim](https://radimrehurek.com/gensim/) has an implementation of Word2Vec that is relatively easy to use. [There's a good tutorial here.](https://rare-technologies.com/word2vec-tutorial/)
* When you're working with vector spaces with high dimensionality and millions of vectors, iterating through your entire space calculating cosine similarities can be a drag. I use [Annoy](https://pypi.python.org/pypi/annoy) to make these calculations faster, and you should consider using it too.